In [1]:
import torch

In [2]:
# 使用.to方法，控制Tensor所在的设备(cpu, cuda)
x = torch.empty(3,3)
if torch.cuda.is_available():
    device = torch.device("cuda")
    y = torch.ones_like(x, device=device)
    x = x.to(device)
    z = x + y
    print(z)
    a=z.to("cpu",torch.double)
    print(a)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)


### 张量，自动求导机制的探索

In [3]:
import torch
# 创建一个张量，指定requires_grad = True，这个属性，用来追踪其计算历史：
x = torch.ones(3, 3, requires_grad=True)
print(x)
# 对张量进行一个简单的一次函数操作：
y = 2*x+1
print(y)
# 再对y进行一个较复杂的函数操作：
z = y*y*3
z_mean = z.mean()
print(z,z_mean)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)
tensor([[3., 3., 3.],
        [3., 3., 3.],
        [3., 3., 3.]], grad_fn=<AddBackward0>)
tensor([[27., 27., 27.],
        [27., 27., 27.],
        [27., 27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


- 以上的requires_grad 属性代表是否需要进行梯度计算。
- grad_fn 表示当前对上一个变量梯度。
- .requires_grad_(...)可以改变当前张量的是否需要进行梯度计算的属性。

In [5]:
## 反向求导：
print(x.grad)

None


In [6]:
## 经过了backward()方法后，torch会计算出x所带有的梯度：
z_mean.backward()
print(x.grad)

tensor([[4., 4., 4.],
        [4., 4., 4.],
        [4., 4., 4.]])


In [7]:
## y是由x创建的，那么y的梯度呢？
print(y.requires_grad)

True


In [8]:
print(y.grad)

None


C:\Users\Administrator\AppData\Local\Temp\ipykernel_10900\1324337507.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\build\aten\src\ATen/core/TensorBody.h:485.)
  print(y.grad)


In [9]:
# 再构造一个例子：
x = torch.randn(3, requires_grad=True)

y = x*2
print(x.shape)

torch.Size([3])


In [10]:
while y.data.norm() < 1000:
    y = y * 2
print(y)

tensor([-1022.8766,   563.6029,    30.1746], grad_fn=<MulBackward0>)


In [12]:
y.backward()
## 可以发现，作为一个向量，其再做反向传播的时候无法直接计算出完整的雅可比行列式
#RuntimeError: grad can be implicitly created only 
#for scalar outputs

RuntimeError: grad can be implicitly created only for scalar outputs

In [14]:
# 这个时候 需要将向量作为参数传入backward：
gradients = torch.tensor([0.1,1.0,0.0001], dtype=torch.float)
y.backward(gradients)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])
